# Projected density of states (PDOS)
For the source code, see [pdos](https://workgraph-collections.readthedocs.io/en/latest/ase/espresso/module.html#workgraph_collections.ase.espresso.pdos.pdos_workgraph).


## Example: Silicon

### Prepare the inputs and submit the workflow



In [1]:
from ase.build import bulk
from aiida import load_profile
from copy import deepcopy

from workgraph_collections.ase.espresso.pdos import PdosWorkgraph

load_profile()

atoms = bulk('Al')


metadata = {
    "options": {
        'prepend_text' : """eval "$(conda shell.posix hook)"
        conda activate aiida
        export OMP_NUM_THREADS=1
        """,
    }
}
pseudopotentials = {"Al": "Al.pbe-n-rrkjus_psl.1.0.0.UPF"}
pseudo_dir = "/home/xing/data/ase/espresso_pseudo"
# pseudo_dir = "/home/wang_x3/datas/pseudos/psl"
scf_input_data = {
    "system": {"ecutwfc": 30, "ecutrho": 240,
               "occupations": "smearing",
                "degauss": 0.01,
                "smearing": "gaussian",
                },
}
nscf_input_data = deepcopy(scf_input_data)
nscf_input_data["system"].update({"occupations": "tetrahedra"})
inputs = {
    "scf": {
        "input_data": scf_input_data,
        "kpts": [10, 10, 10],
        "computer": "localhost",
        "metadata": metadata
    },
    "nscf": {
        "input_data": nscf_input_data,
        "kpts": [20, 20, 20],
        "computer": "localhost",
        "metadata": metadata
    },
    "dos": {
        "input_data": {},
        "computer": "localhost",
        "metadata": metadata
    }
}
#------------------------- Set the inputs -------------------------
wg = PdosWorkgraph.build(atoms=atoms,
                    pw_command="mpirun -np 4 pw.x",
                    dos_command="mpirun -np 4 dos.x",
                    projwfc_command="mpirun -np 4 projwfc.x",
                    pseudopotentials=pseudopotentials,
                    pseudo_dir=pseudo_dir,
                    inputs=inputs)
wg.to_html()

Run the calculation


In [2]:
wg.run()

09/12/2025 01:39:40 PM <790230> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [186604|WorkGraphEngine|continue_workgraph]: tasks ready to run: pw_calculator
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[186604|WorkGraphEngine|continue_workgraph]: tasks ready to run: pw_calculator
09/12/2025 01:39:41 PM <790230> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [186604|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 186606
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[186604|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 186606
09/12/2025 01:39:47 PM <790230> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [186604|WorkGraphEngine|update_task_state]: Task: pw_calculator, type: PYTHONJOB, finished.
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[186604|WorkGraphEngine|update_task_state]: Task: pw_calculator, type: PYTHONJOB, finishe

{}

## Print the results

In [3]:
import matplotlib.pyplot as plt
import numpy as np

#------------------------- Print the output -------------------------
dos_data = wg.outputs.dos.dos.value.value
pdos_data = wg.outputs.projwfc.projections.value.value
# make plots
plt.figure(figsize = (8, 4))
plt.plot(dos_data['x']['array'], dos_data['y']['arrays'][1], label='total')
# plt.plot(pdos_data['totol']['energy'], pdos_data['totol']['dos'], label='total')
# for file_name, data in pdos_data['atom'].items():
        # value = np.array(data)
        # plt.plot(value[:, 0], np.sum(value[:, 2:], axis=1), label=file_name[-4:])
plt.yticks([])
plt.xlabel('Energy (eV)')
plt.ylabel('DOS')
plt.legend(frameon=False)


AttributeError: 'NoneType' object has no attribute 'value'